#### Processing Data from Magnetic Tweezers Force-Cycles Assays
Related to: "DNA-driven dimerization of Ku70/Ku80 regulates the synapsis of double strand breaks by APLF."

This notebook processes DNA length reduction data obtained from magnetic tweezers (MT) force cycles assays.

MT data can be downloaded from Zenodo, doi: 10.5281/zenodo.15609892

In [ ]:
# Import libraries and helper functions
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy.optimize import curve_fit
from scipy.stats import gaussian_kde
import functions

Choose the project folder and define the conditions to import:

In [ ]:
project = os.getcwd()  # data parent folder to data, user defined
# ----
experiments_log = []

In [ ]:
condition = '60bp'
# results = [[experiment-date, flowcell-number, results-file-number, [beads_to_discard]]]
results = [['2025_01_27', '#1', 1, [1,9,26,31,32,33]], 
           ['2025_01_27', '#2', 4, [2,12,18,20]], 
           ['2025_01_27', '#3', 7, [18,19]]] 

experiments_log = functions.add_results_to_experiments_log(experiments_log, condition, results)

In [ ]:
condition = '60bp+Ku'
results = [['2025_02_05', '#1', 5, [1,2]], 
           ['2025_02_05', '#2', 7, []], 
           ['2025_02_05', '#3', 9, []]]

experiments_log = functions.add_results_to_experiments_log(experiments_log, condition, results)

In [ ]:
condition = '60bp+Ku+APLF'
results = [['2025_01_27', '#1', 2, [1,9,26,31,32,33]], 
           ['2025_01_27', '#2', 5, [2,12,15,18,20]], 
           ['2025_01_27', '#3', 8, [18,19]]]

experiments_log = functions.add_results_to_experiments_log(experiments_log, condition, results)

In [ ]:
condition = '60bp+Ku+APLF(PBZ)mutant'
results = [['2025_07_18', '#2', 5, [36,32,26]], 
           ['2025_07_18', '#3', 8, [0,14,30,40,7,17,33,36]], 
           ['2025_07_18', '#4', 10, [6,14,28,32]]]

experiments_log = functions.add_results_to_experiments_log(experiments_log, condition, results)

In [ ]:
condition = '6020bp'
results = [['2025_01_29', '#1', 1, []], 
           ['2025_01_29', '#2', 4, [20]], 
           ['2025_01_29', '#3', 7, [10,30]]]

experiments_log = functions.add_results_to_experiments_log(experiments_log, condition, results)

In [ ]:
condition = '6020bp+Ku+APLF'
results = [['2025_01_29', '#1', 2, []], 
           ['2025_01_29', '#2', 5, [20,11]], 
           ['2025_01_29', '#3', 8, [10,30,9]]]

experiments_log = functions.add_results_to_experiments_log(experiments_log, condition, results)

In [ ]:
condition = '20bp'
results = [['2025_01_27', '#4', 10, [1,7,11,14]], 
           ['2025_01_27', '#5', 13, [1,11]], 
           ['2025_01_27', '#6', 17, [8,10,21,27,29]]]

experiments_log = functions.add_results_to_experiments_log(experiments_log, condition, results)

In [ ]:
condition = '20bp+Ku+APLF'
results = [['2025_01_27', '#4', 11, [1,5,7,11,14]], 
           ['2025_01_27', '#5', 14, [1,11]], 
           ['2025_01_27', '#6', 18, [8,10,21,27,29]]]

experiments_log = functions.add_results_to_experiments_log(experiments_log, condition, results)

In [ ]:
# summary
experiments_log

Notes on nomenclature:
- `z` refers to displacement along the vertical axis
- variables containing `step` correspond to a single experimental step
- thresholds are defined in physical units whenever possible

#### Plot and export all timecourses (including beads to discard)

In [ ]:
# IMPORTANT: This takes time. Be patient.
for line in experiments_log:
    cond, date, flowcell, file, _ = line
    figpath = os.path.join(project, date, 'data_timecourses')
    if not os.path.exists(figpath): os.mkdir(figpath)
    path = os.path.join(project, date, f'resultados_{file}_PAR.dat')
    data = pd.read_csv(path, decimal='.', delimiter='\t', header=0)
    beads = [int(label.split('-')[0].split('d')[-1]) for label in data.columns if 'Bead' in label and 'Z' in label]
    for b in beads:
        label = f'R#{file}B#{b}_{cond}'
        baseline = data[f'Bead{b}-Z(um)'][data['Step']==0].mean()
        y = data[f'Bead{b}-Z(um)']
        yalign = data[f'Bead{b}-Z(um)'] - baseline
        x = data['Time (sg)']
        fig, ax = plt.subplots()
        #ax.scatter(x, y, s=1, color='blue', alpha=0.7)
        ax.scatter(x, yalign, s=1, color='red', alpha=0.7)
        ax.set(xlabel='Time (s)', xlim=(x.min(), x.max()), ylabel='Length (um)', ylim=(-1.59, 0.19))
        ax.grid(color='lightgrey', ls='--')
        fig.savefig(os.path.join(figpath, f'{label}.png'), transparent=False, dpi=100)
        plt.close(fig)

#### Visualize a single timecourse

In [ ]:
expected_reductions = [-174.68, -186.96, -199.24]
#cond, date, file, bead, draw_line = ['60bp+Ku+APLF', '2025_01_27', 2, 5, -174.68]
#cond, date, file, bead, draw_line = ['6020bp+Ku+APLF', '2025_01_29', 5, 10, -186.96]
#cond, date, file, bead, draw_line = ['20bp+Ku+APLF', '2025_01_27', 18, 7, -199.24]
cond, date, file, bead, draw_line = ['60bp+Ku+APLF(PBZ)mutant', '2025_07_18', 5, 9, -174.68]

# ----------------------------------------------------------------
path = os.path.join(project, date, f'resultados_{file}_PAR.dat')
data = pd.read_csv(path, decimal='.', delimiter='\t', header=0)
z = data[f'Bead{bead}-Z(um)']
baseline = z[data['Step']==0].mean()
t = data['Time (sg)']
mag = data['Shift pos (mm)']
f = functions.force_equation(mag)
# -----------------------------------------------------------------
plt.close('all')
fig, ax = plt.subplots(figsize=(5,2), nrows=2, ncols=2, sharey='row', sharex='col',
                        gridspec_kw={'width_ratios':[5,1], 'height_ratios':[4,1], 'top':.91, 'bottom':.2, 'hspace':0.025, 'wspace':0.0115, 'left':.2, 'right':.99})

ax[1,1].remove()

ax[0,0].scatter(t, z-baseline, s=0.5, color='blue', alpha=0.2)
ax[0,0].set(ylabel='Δz (µm)', ylim=(-1.59, 0.19))

ax[1,0].plot(t,f, color='grey')
ax[1,0].set(xlabel='Time (s)', xlim=(0, t.max()), ylabel='F (pN)', ylim=(0, 2.5))

bw = 0.05
x_range = (-0.1,5, 0.25)
bins = np.arange(min(z-baseline), max(z-baseline), bw)
bin_centers = (bins[:-1] + bins[1:]) / 2
counts, bin_edges = np.histogram(z-baseline, bins=bins, range=x_range, density=True)
ax[0,1].hist(bin_centers, bins, weights=counts, color='blue', edgecolor='white', linewidth=0.5, orientation='horizontal')
ax[0,1].set(xlim=(0, 3)) # xlabel='Density'

for axis in [ax[0,0], ax[0,1]]:
    axis.axhline(y=0, color='k', linestyle='--', linewidth=1, alpha=0.7)
    axis.axhline(y=draw_line/1000, color='k', alpha=0.7, linestyle='--', linewidth=1)
    #axis.annotate(f'{draw_line} nm', (50, (draw_line-150)/1000), fontsize=8) if axis == ax[0,0] else None

# grid
for axis in (ax[0,0], ax[1,0], ax[0,1]):
    axis.grid(color = 'lightgrey', linestyle = ':', linewidth = 1)
    axis.set_axisbelow(True)
ax[0,0].set(ylim=(-1.45, 0.19))

figpath = os.path.join(project, date, f'{date}_{cond}_R#{file}_B#{bead}.png')

In [ ]:
# save the figure
fig.savefig(figpath, transparent=False, dpi=300)

#### Define params and functions to compute plateaux

In [ ]:
rolling_window = 30     # number of frames used for signal smoothing
diff_threshold_um = 0.0025  # flatness threshold [µm] for plateau detection
time_minimum_ms = 200       # minimum time required to consider a plateau [ms]

#### Plot and export all events individually

In [ ]:
# IMPORTANT:
# Plotting and exporting all events takes tens of minutes. Be patient.

figpath = os.path.join(project, 'individual-events-by-condition')
if not os.path.exists(figpath): os.mkdir(figpath)

for line in experiments_log:
    cond, date, flowcell, file, discard = line

    figpath = os.path.join(project, 'individual-events-by-condition', cond)
    if not os.path.exists(figpath): os.mkdir(figpath)

    path = os.path.join(project, date, f'resultados_{file}_PAR.dat')
    data = pd.read_csv(path, decimal='.', delimiter='\t', header=0)
    
    beads = [int(label.split('-')[0].split('d')[-1]) for label in data.columns if 'Bead' in label and 'Z' in label]
    for d in discard: beads.remove(d)

    time = data['Time (sg)']
    force = functions.force_equation(data['Shift pos (mm)'])
    steps = data['Step']

    for bead in beads:
        label = f'{cond}_{date}_R#{file}_B#{bead}'
        baseline = data[f'Bead{bead}-Z(um)'][steps==0].mean()
        z = data[f'Bead{bead}-Z(um)']-baseline
        functions.plot_and_export_individual_events_with_details(z, time, force, steps, label, diff_threshold_um, time_minimum_ms, rolling_window, figpath)
print('Done, all events have been exported.')

#### Extract events' deltas and lifetimes

In [ ]:
# exclude extended events or non-intramolecule interactions?
exclude_extended = True
exclude_big_reductions = True
# ------------
# save the data for technical repeats seperatly?
save_reps_seperatly = False
# -----------
frame_rate = 120    # experimenal value, frames/s
frames_threshold = int(frame_rate * (time_minimum_ms*10**(-3)))
dataset={}
for line in experiments_log:
    cond, date, flowcell, file, discard = line
    deltas_all = []
    lifetimes_all = []
    
    path = os.path.join(project, date, f'resultados_{file}_PAR.dat')
    data = pd.read_csv(path, decimal='.', delimiter='\t', header=0)
    
    beads = [int(label.split('-')[0].split('d')[-1]) for label in data.columns if 'Bead' in label and 'Z' in label]
    for d in discard: beads.remove(d)

    molecules = len(beads)
    molecules_with_events = 0
    molecules_with_strong_events = 0

    time = data['Time (sg)']
    force = functions.force_equation(data['Shift pos (mm)'])
    steps = data['Step']

    for bead in beads:
        label = f'{cond}_{date}_R#{file}_B#{bead}'
        baseline = data[f'Bead{bead}-Z(um)'][steps==0].mean()
        z = data[f'Bead{bead}-Z(um)']-baseline
        events = False # assume molecule with no events
        strong_events =  False # assume molecule with no events

        for step in range(2, len(steps.unique())+1, 2):        
            time_defined_step = time[steps==step]
            z_defined_step = z[steps==step]
            try: smoothed, diff, zplateaux, lifetimes = functions.process_step(z_defined_step, time_defined_step, diff_threshold_um, frames_threshold, rolling_window)
            except: pass

            # Define filtering conditions
            filters = []
            if exclude_extended: filters.append(lambda z: z < -0.05)
            if exclude_big_reductions: filters.append(lambda z: z > -0.325)

            # Apply filters if any exist
            for condition in filters:
                filtered = [(z, t) for (z, t) in zip(zplateaux, lifetimes) if condition(z)]
                zplateaux, lifetimes = zip(*filtered) if filtered else ([], [])
                zplateaux, lifetimes = list(zplateaux), list(lifetimes)
            
            deltas_all += zplateaux
            lifetimes_all += lifetimes
            if len(zplateaux) > 0: events = True
            if any(num > 0.90 for num in lifetimes): strong_events = True # seconds
        if events: molecules_with_events += 1
        if strong_events: molecules_with_strong_events += 1

    if save_reps_seperatly:
        cond = f'{cond}_{date}_R#{file}'

    if cond in dataset: 
        deltas_all = dataset[cond]['Plateaux'] + deltas_all
        lifetimes_all = dataset[cond]['LifetimesNorm'] + lifetimes_all
        molecules = dataset[cond]['Molecules'] + molecules
        molecules_with_events = dataset[cond]['Molecules_w_events'] + molecules_with_events
        molecules_with_strong_events = dataset[cond]['Molecules_w_strong_events'] + molecules_with_strong_events

    dataset[cond] = {'Plateaux': deltas_all, 
                     'LifetimesNorm': lifetimes_all,
                     'Molecules': molecules,
                     'Molecules_w_events': molecules_with_events,
                     'Molecules_w_strong_events': molecules_with_strong_events}

In [ ]:
dataset.keys()

#### Scatter plots -- Reductions versus lifetimes of events

In [ ]:
# define the conditions and order you want to plot them: (same_DNA, refers to the branch configuration)

# ----- for global conditions
keys_to_plot, same_DNA = ['60bp+Ku+APLF','20bp+Ku+APLF','6020bp+Ku+APLF','60bp+Ku+APLF(PBZ)mutant'], False

# ----- for individual repetitions within conditions
#keys_to_plot, same_DNA = list(dataset.keys()), True

In [ ]:
extra_line = False
color = 'darkblue'
plt.close('all')
fig, axs = plt.subplots(figsize=(3.5*len(keys_to_plot),4), nrows=2, ncols=2*len(keys_to_plot), sharex='col', sharey='row',
                        gridspec_kw={'height_ratios':[0.9, 3.0], 'width_ratios':len(keys_to_plot)*[1.0, 0.3],
                                     'top':.91, 'bottom':.2, 'hspace':0.05, 'wspace':0.04, 'left':.1, 'right':.90})

# Define a Gaussian function
def gaussian(x, A, mu, sigma):
    return A * np.exp(-(x - mu)**2 / (2 * sigma**2))

# positions for annotations
xpos, ypos = 0.02, -330

for n, (i, key) in enumerate(zip(range(0, 2*len(keys_to_plot), 2), keys_to_plot)):
    dz = np.array(dataset[key]['Plateaux'])*1000
    life = np.array(dataset[key]['LifetimesNorm'])

    axs[0,i].set_title(key, fontsize=10)
    axs[0,i+1].remove()

    for axis in [axs[0,i], axs[1,i], axs[1,i+1]]:
        axis.grid(color='lightgrey', ls='--', zorder=5)
        axis.set_axisbelow(True)

    if same_DNA: expected_reductions = len(keys_to_plot)*[-174.68]
    else: expected_reductions = [-174.68, -186.96, -199.24, -174.68] # remove last one if mutant not present
    for axis in [axs[1,i], axs[1,i+1]]:
        axis.axhline(y=0, color='k', linestyle='--', linewidth=1)
        axis.axhline(y=expected_reductions[n], color='black', alpha=1, linestyle='--', linewidth=1)
        axis.annotate(f'{expected_reductions[n]} nm', (0.02, expected_reductions[n]-20), fontsize=8) if axis == axs[1,i] else None
        if extra_line: 
            extra = round(expected_reductions[n]+100, 2)
            c = 'red'
            axis.axhline(y=extra, color=c, alpha=1, linestyle='--', linewidth=1)
            axis.annotate(f'{extra} nm', (0.02, (extra)-20), fontsize=8, color=c) if axis == axs[1,i] else None
    
    # lifetime histogram
    density = True
    bw = 0.03; max_t = 10
    t_range = (0, 1)
    bins = np.arange(t_range[0], t_range[1]+bw, bw)
    bin_centers = (bins[:-1] + bins[1:]) / 2
    counts, bin_edges = np.histogram(life, bins=bins, range=t_range, density=density)
    axs[0,i].hist(bin_centers, bins, weights=counts, color=color, edgecolor='white', alpha=1, linewidth=1, orientation='vertical')
    if density: axs[0,i].set(xlim=(-0.05,1.05), ylim=(0.001, max_t), ylabel='t density')
    else: axs[0,i].set(xlim=(-0.05,1.05), ylim=(0.001, None), ylabel='t counts')
    #axs[0,i].set_yscale('log'); max_t = 40
    
    # length reduction histogram
    density = False
    bw = 10
    reductions_range = (-350, 50)
    try:
        bins = np.arange(min(dz), max(dz), bw)
        bin_centers = (bins[:-1] + bins[1:]) / 2
        counts, bin_edges = np.histogram(dz, bins=bins, range=reductions_range, density=density)
        axs[1,i+1].hist(bin_centers, bins, weights=counts, color=color, edgecolor='white', alpha=1, linewidth=1, orientation='horizontal')

        if key == '20bp+Ku+APLF' or key == '60bp+Ku+APLF(PBZ)mutant': pass
        else:
            # Fit the data
            initial_guess = [1, expected_reductions[n], 1]  # [A, mu, sigma]
            constrains=([0, -np.inf, 0], [np.inf, np.inf, np.inf])
            popt, pcov = curve_fit(gaussian, bin_centers, counts, p0=initial_guess, bounds=constrains)
            # plot the fitting
            A_fit, mu_fit, sigma_fit = popt
            extension_range = np.linspace(reductions_range[0], reductions_range[1], 500)
            axs[1,i+1].plot(gaussian(extension_range, *popt), extension_range, label='Gaussian fit', color='red', linewidth=1, ls='--')
            print(f"{key}_Fitted parameters: A = {A_fit:.2f}, mu = {mu_fit:.2f}, sigma = {sigma_fit:.2f}, fwhm={(2.35*sigma_fit):.2f}")
            axs[1,i+1].annotate(f'μ = {mu_fit:.2f}', (xpos+150, ypos+240), fontsize=8, color='red', ha='right')
            axs[1,i+1].annotate(f'σ = {sigma_fit:.2f}', (xpos+150, ypos+210), fontsize=8, color='red', ha='right')
        
    except: pass
    if density: axs[1,i+1].set(xlim=(0.00001, 0.026), ylim=(-350, 50), xlabel='dz density')
    else: axs[1,i+1].set(xlim=(0, 151), ylim=(-350, 50), xlabel='dz counts')

    # scatter plot
    # Calculate the point density
    try:
        xy = np.vstack([life,dz])
        vmin, vmax = 0, 0.015
        z = gaussian_kde(xy)(xy)
        density = axs[1,i].scatter(life, dz, s=10, alpha=1, c=z, cmap='cividis')#, vmin=vmin, vmax=vmax)
    except: 
        axs[1,i].scatter(life, dz, s=10, alpha=1, c=color)
    
    # annotations
    axs[1,i].annotate(f'{dataset[key]["Molecules"]} molecules (x20 steps)', (xpos, ypos+40), fontsize=8)
    axs[1,i].annotate(f'{dataset[key]["Molecules_w_events"]} molecules w/events', (xpos, ypos+20), fontsize=8)
    axs[1,i].annotate(f'{len(dz)} total #events', (xpos, ypos), fontsize=8)

    # adjust limits and general details
    axs[1,i].set(xlim=(-0.05,1.05), ylim=(-350, 50), xlabel='Lifetime, tau (step ratio)')
    if i==0: axs[1,i].set(ylabel='Length reduction, dz (nm)')
try: fig.colorbar(density, ax=[axs[1,-1]], label='KDE density', orientation='vertical', pad=0.1)
except: pass

In [ ]:
# save fig
fig.savefig(os.path.join(project, f'scatter-plots.png'))

#### Bar plots -- Fraction of strong events 

In [ ]:
# To plot the fraction of strong events with respect to the total number os events you must calculate deltas and lifetimes for individual repetitios by 
# defining --> save_reps_seperatly = True

cond1 = '60bp+Ku+APLF', -174.68, ['60bp+Ku+APLF_2025_01_27_R#2', '60bp+Ku+APLF_2025_01_27_R#5', '60bp+Ku+APLF_2025_01_27_R#8']
cond2 = '20bp+Ku+APLF', -186.96, ['20bp+Ku+APLF_2025_01_27_R#11', '20bp+Ku+APLF_2025_01_27_R#14', '20bp+Ku+APLF_2025_01_27_R#18']
cond3 = '6020bp+Ku+APLF', -199.24, ['6020bp+Ku+APLF_2025_01_29_R#2', '6020bp+Ku+APLF_2025_01_29_R#5', '6020bp+Ku+APLF_2025_01_29_R#8']
cond4 = '60bp+Ku+APLF(PBZ)mutant', -174.68, ['60bp+Ku+APLF(PBZ)mutant_2025_07_18_R#5', '60bp+Ku+APLF(PBZ)mutant_2025_07_18_R#8', '60bp+Ku+APLF(PBZ)mutant_2025_07_18_R#10']

In [ ]:
data_to_plot = [cond1, cond2, cond3]
#data_to_plot = [cond1, cond4]

tags = []
events_percentages_all = []
events_means = []
events_stds = []

strong_percentages_all = []
strong_means = []
strong_stds = []

for cond in data_to_plot:
    tag, reduct, keys = cond
    tags += [tag]
    events_percentage = []
    strong_percentage = []
    
    # Define filters
    lower_limit, higher_limit = reduct-50, reduct+50
    filters = []
    filters.append(lambda z,t: z < higher_limit)
    filters.append(lambda z,t: z > lower_limit)
    filters.append(lambda z,t: t > 0.5)

    for key in keys:
        dz = np.array(dataset[key]['Plateaux'])*1000
        life = np.array(dataset[key]['LifetimesNorm'])

        positive = len(dz)
        potential = dataset[key]['Molecules']*20 # 20 cycles per molecule
        events_percentage += [(positive/potential)*100]

        filt_dz = dz[dz <= higher_limit]
        filt_dz = filt_dz[filt_dz >= lower_limit]
        diff_dz = positive - len(filt_dz)

        filt_t = life[life >= 0.9]
        diff_t = positive - len(filt_t)

        strong = positive - diff_dz - diff_t
        strong_percentage += [(strong/potential)*100]
    
    events_means += [np.mean(events_percentage)]
    events_stds += [np.std(events_percentage)]

    strong_means += [np.mean(strong_percentage)]
    strong_stds += [np.std(strong_percentage)]

    events_percentages_all += [events_percentage]
    strong_percentages_all += [strong_percentage]

In [ ]:
fig, ax = plt.subplots()

w = 0.8    # bar width
x = range(1,len(tags)+1) # x-coordinates of your bars

ax.bar(x, height=events_means, yerr=events_stds, capsize=12, width=w, tick_label=tags, color='lightsteelblue', alpha=0.7, label='Total events')
ax.bar(x, height=strong_means, yerr=strong_stds, capsize=12, width=w, tick_label=tags, color='cornflowerblue', alpha=1, label='Strong and specific events')

for i, y in enumerate(events_percentages_all):
    ax.scatter([x[i]-0.05, x[i], x[i]+0.05], y, color='black', s=10)

for i, y in enumerate(strong_percentages_all):
    ax.scatter([x[i]-0.05, x[i], x[i]+0.05], y, color='black', s=10)

ax.set(ylim=(0,100), ylabel='%')
ax.grid(color='0.90', ls='--')
ax.set_axisbelow(True)
ax.legend(loc='upper right')

for cond, mean_total, std_total, mean_strong, std_strong in zip(tags, events_means, events_stds, strong_means, strong_stds):
    print(f'{cond}, total events: mean={mean_total}, std={std_total}')
    print(f'{cond}, strong events: mean={mean_strong}, std={std_strong}')

In [ ]:
# save
fig.savefig(os.path.join(project, f'comparison-mutant-APLF.svg'))